In this notebook, we demonstrate how to extract and lookup for contextually-most-similar words using BERT and nearest neighbor search. 

This was inspired by the StackOverflow question https://stackoverflow.com/questions/59865719/how-to-find-the-closest-word-to-a-vector-using-bert

Note: The environment we use is google colab, first run. Subsequent runs after reset have had issues with dependencies.

# learn to extract embeddings from bert

We use `bert-embedding` package; see https://pypi.org/project/bert-embedding/

We use GPU, so please choose the Colab kernel accordingly

In [ ]:
!pip install mxnet-cu102
!pip install bert-embedding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 380.8 MB 6.1 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.6 MB 103.5 MB/s 
     |████████████████████████████████| 13.8 MB 62.5 MB/s 
     |████████████████████████████████| 209 kB 52.5 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.6.0-py3-none-any.whl size=259930 sha256=8240234c55c300214ba60c5a84300722430138fac2d524b17e83cebf261b7c7a
  Stored in directory: /root/.cache/pip/wheels/a6/41/8f/45bd1c58055d87aee5a71b6756a427ea8d92e506b3a9d17370
Successfully built gluonnlp
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      

In [ ]:
import mxnet as mx
from bert_embedding import BertEmbedding

In [ ]:
# ctx = mx.gpu(0)
bert = BertEmbedding()

Vocab file is not found. Downloading.


In [ ]:
from tqdm.auto import tqdm, trange

In [ ]:
bert_abstract = """We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers.
 Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations by jointly conditioning on both left and right context in all layers.
 As a result, the pre-trained BERT representations can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications. 
BERT is conceptually simple and empirically powerful. 
It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE benchmark to 80.4% (7.6% absolute improvement), MultiNLI accuracy to 86.7 (5.6% absolute improvement) and the SQuAD v1.1 question answering Test F1 to 93.2 (1.5% absolute improvement), outperforming human performance by 2.0%."""

In [ ]:
sentences = bert_abstract.split('\n')
result = bert(sentences)
toks, embs = result[0]
print(toks)
print(len(toks), len(embs))
print(embs[0][:10])

['we', 'introduce', 'a', 'new', 'language', 'representation', 'model', 'called', 'bert', ',', 'which', 'stands', 'for', 'bidirectional', 'encoder', 'representations', 'from', 'transformers']
18 18
[ 0.47964773  0.1824888  -0.28597528 -0.46567446  0.01248981 -0.07430505
 -0.18017295  0.37813222  0.9135139  -0.25295883]


# process a corpus

We download a 10k web-public .com corpus from https://wortschatz.uni-leipzig.de/en/download/


In [ ]:
!wget https://files.pushshift.io/gab/GABPOSTS_2018-10.xz

--2022-11-27 04:12:55--  https://files.pushshift.io/gab/GABPOSTS_2018-10.xz
Resolving files.pushshift.io (files.pushshift.io)... 104.21.28.11, 172.67.170.36, 2606:4700:3030::ac43:aa24, ...
Connecting to files.pushshift.io (files.pushshift.io)|104.21.28.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337144496 (322M) [application/octet-stream]
Saving to: ‘GABPOSTS_2018-10.xz’

GABPOSTS_2018-10.xz 100%[===================>] 321.53M  17.3MB/s    in 20s     

2022-11-27 04:13:15 (16.4 MB/s) - ‘GABPOSTS_2018-10.xz’ saved [337144496/337144496]



In [ ]:
import lzma
import json
import pandas as pd
import re
import numpy as np
gab_posts = pd.DataFrame()
temp = []
counter = 0
with lzma.open('GABPOSTS_2018-10.xz', mode='r') as file:
    for line in file:
      # Can add raw/other fields, just worried about memory requirements
      # I think I did this in a cleaner way before, but w/e it works :)
      if counter > 36000:
        break
      counter = counter + 1
      temp.append({"body": re.sub(r"(?:\@|\\|https?\://)\S+", "",json.loads(line)['body']), "date":json.loads(line)['created_at']})


In [ ]:
gab_posts = gab_posts.append(temp)

In [ ]:
len(temp)

In [ ]:
banned_words = ['pussie', 'phonesex', 'footjob', 'horniest', 'clitoris', 'headfuck', 'areola', 'pussies', 'goddamnes', 'suicide', 'voyeurweb', 'suicide girls', 'niggarded', 'deepthroat', 'fuckbuddy', 'nigra', 'freefuck', 'boob', 'hentai', 'rentafuck', 'wanking', 'jerk off', 'molester', 'horney', 'titfuckin', 'milf', 'wrapping men', 'whorefucker', 'masturbating', 'dick', 'honkers', 'chocolate rosebuds', 'neonazi', 'vibrater', 'uptheass', 'shitdick', 'pussycat', 'naked', 'group sex', 'suckmydick', 'pussyeater', 'masturbate', 'stupidfuck', 'nig', 'rape', 'meth', 'virgin', 'livesex', 'terrorist', 'upskirt', 'shortfuck', 'genital', 'jiggaboo', 'marijuana', 'cumshots', 'koon', 'holestuffer', 'tit', 'assbagger', 'ball sack', 'sexpot', 'suckmyass', 'lovejuice', 'phukking', 'wigger', 'black cock', 'whiskeydick', 'blonde on blonde action', 'retarded', 'kunt', 'motherfuckin', 'orgy', 'ejaculation', 'fuckme', 'phone sex', 'fuckher', 'niggerhole', 'intercourse', 'pussylips', 'niggardly', 'tongethruster', 'nig nog', 'kumbullbe', 'nigger', 'wanker', 'peepshpw', 'cocks', 'omorashi', 'female squirting', 'blow job', 'bung hole', 'homicide', 'penetration', 'puddboy', 'gang bang', 'lickme', 'spermhearder', 'titties', 'rigger', 'shitblimp', 'twat', 'fag', 'gangbanger', 'orgasim', 'porno', 'assfuck', 'pussy', 'sodomy', 'cumshot', 'cock', 'jihad', 'niggaz', 'picaninny', 'bondage', 'dry hump', 'poorwhitetrash', 'whitenigger', 'nip', 'masturbation', 'peni5', 'sexed', 'escort', 'g-spot', 'muffindiver', 'fingerbang', 'shite', 'gypo', 'scrotum', 'creampie', 'goddamnmuthafucker', 'foreskin', 'titty', 'dildo', 'sexkitten', 'anus', 'niggling', 'niggerhead', 'footlicker', 'pussylover', 'limpdick', 'fucktard', 'male squirting', 'gangbang', 'nigg', 'suckdick', 'vagina', 'reestie', 'bangbros', 'givehead', 'spank', 'trailertrash', 'giant cock', 'fucktards', 'sexo', 'pussypounder', 'gaymuthafuckinwhore', 'negroid', 'lsd', 'ball gag', 'jigga', "nigger's", 'orgasm', 'nlgger', 'asskiss', 'coprolagnia', 'boobs', 'pussylicker', 'whitetrash', 'mothafuckings', 'fingering', 'scum', 'paedophile', 'sperm', 'testicle', 'poopchute', 'wank', 'jerkoff', 'octopussy', 'pedophile', 'reverse cowgirl', 'negroes', 'suckmytit', 'big tits', 'sonofbitch', 'swastika', 'jizz', 'sexslave', 'bunghole', 'retard', 'hore', 'nipplering', 'kink', 'nipples', 'vaginal', 'tittie', 'hitler', 'jiggabo', 'pedobear', 'handjob', 'pubic', 'kkk', 'niggled', 'pthc', "negro's", 'doggystyle', 'samckdaddy', 'gangbanged', 'clit', 'hand job', 'beaners', 'ecchi', 'doggy style', 'nutten', 'bdsm', 'cunnilingus', 'killing', 'genitals', 'poop chute', 'fuckfest', 'spermherder', 'brunette action', 'motherfuck', 'cumming', 'erotic', 'splooge moose', 'foursome', 'niglet', 'nigre', 'incest', 'cunt', 'molest', 'threesome', 'kissass', 'narcotic', 'sexhouse', 'nudity', 'fudgepacker', 'snownigger', 'white power', 'jiggerboo', 'honky', 'rosy palm and her 5 sisters', 'nittit', 'horny', 'hotpussy', 'ball sucking', 'nignog', 'palesimian', 'jizjuice', 'zoophilia', 'nigga', 'asslicker', 'niggle', 'nlggor', 'pornography', 'sexing', 'slutt', 'titlicker', 'kunnilingus', 'fuckwhore', 'wet dream', 'spunk', 'pisser', 'puss', 'boner', 'skeet', 'sextoys', 'vibrator', 'manpaste', 'faggot', 'humping', 'nipple', 'double penetration', 'coons', 'assklown', 'pubes', 'fuckface', 'anal', 'nimphomania', 'blowjob', 'rimjob', 'fisting', 'niggardliness', 'sultry women', 'jizzim', 'kinkster', 'skankfuck', 'penis', 'how to kill', 'semen', 'mothafucker', 'analsex', 'niggur', 'panty', 'deep throat', 'foot fetish', 'freakyfucker', 'date rape', 'assblaster', 'bukkake', 'lesbo', 'spaghettinigger', 'beaner', 'clover clamps', 'twobitwhore', 'nigr', 'fuckfriend', 'sextoy', 'prostitute', 'pussyfucker', 'kanake', 'porchmonkey', 'testicles', 'erotism', 'pusy', 'assjockey', 'pimpjuic', 'booty call', 'kaffir', 'fuckable', 'goldenshower', 'homobangers', 'pegging', 'rapist', 'venus mound', 'raping', 'fudge packer', 'sexcam', 'timbernigger', 'viagra', 'make me come', 'beastiality', 'leather restraint', 'coon', 'futanari', 'fuckina', 'iblowu', 'masterbate', 'luckycammeltoe', "niggardliness's", 'fuckmehard', 'tits', 'suckme', 'intheass', 'niggarding', 'tonguetramp', 'niggor', 'schlong', 'niggah', 'raped', 'nazi', 'two girls one cup', 'huge fat', 'upthebutt', 'daterape', 'mastabater', 'cum', 'asslick', 'raghead', 'bestiality', 'golden shower', 'niggers', 'penises', 'mufflikcer', 'camel toe', 'shaved pussy', 'niggles', 'jijjiboo']


In [ ]:

gab_posts['body'].replace(r'\n',' ', regex=True, inplace=True)
gab_posts['body'].replace(r'\r',' ', regex=True, inplace=True)
gab_posts['body'].replace('', np.nan, inplace=True)
gab_posts.dropna(subset=['body'], inplace=True)
gab_posts['body'].str.lower()


0                                       #trade #winning   
1        o deputado arthur lira (pp/al):  ⚠️ responde a...
2                      cocaine mitch comes out swinging.  
3        #demoncrats have no redeeming value. a black c...
4        putting up the rent in one town/city/country w...
                               ...                        
35996    wow, you have some pent up anger it seems. don...
35997                     see ya , wouldn't want to be ya.
35998           wieder hat die antifa rassisten entlarvt. 
35999         is she /he one of those transgender mutants?
36000                  the normies they make vader afraid.
Name: body, Length: 31807, dtype: object

remove row index from each sentence

In [ ]:
all_sentences = gab_posts['body'].to_numpy()

# create a search index

In [ ]:
from sklearn.neighbors import KDTree
import numpy as np


class ContextNeighborStorage:
    def __init__(self, sentences, model):
        self.sentences = sentences
        self.model = model

    def process_sentences(self):
        result = self.model(self.sentences)

        self.sentence_ids = []
        self.token_ids = []
        self.all_tokens = []
        all_embeddings = []
        for i, (toks, embs) in enumerate(tqdm(result)):
            for j, (tok, emb) in enumerate(zip(toks, embs)):
                self.sentence_ids.append(i)
                self.token_ids.append(j)
                self.all_tokens.append(tok)
                all_embeddings.append(emb)
        all_embeddings = np.stack(all_embeddings)
        # we normalize embeddings, so that euclidian distance is equivalent to cosine distance
        self.normed_embeddings = (all_embeddings.T / (all_embeddings**2).sum(axis=1) ** 0.5).T

    def build_search_index(self):
        # this takes some time
        self.indexer = KDTree(self.normed_embeddings)

    def query(self, query_sent, query_word, k=10, filter_same_word=False):
        toks, embs = self.model([query_sent])[0]

        found = False
        for tok, emb in zip(toks, embs):
            if tok == query_word:
                found = True
                break
        if not found:
            raise ValueError('The query word {} is not a single token in sentence {}'.format(query_word, toks))
        emb = emb / sum(emb**2)**0.5

        if filter_same_word:
            initial_k = max(k, 100)
        else:
            initial_k = k
        di, idx = self.indexer.query(emb.reshape(1, -1), k=initial_k)
        distances = []
        neighbors = []
        contexts = []
        for i, index in enumerate(idx.ravel()):
            token = self.all_tokens[index]
            if filter_same_word and (query_word in token or token in query_word):
                continue
            distances.append(di.ravel()[i])
            neighbors.append(token)
            contexts.append(self.sentences[self.sentence_ids[index]])
            if len(distances) == k:
                break
        return distances, neighbors, contexts

Now let's use this indexer

In [ ]:
storage = ContextNeighborStorage(sentences=all_sentences, model=bert)
storage.process_sentences()

  0%|          | 0/31807 [00:00<?, ?it/s]

Creating the index would require some time

In [ ]:
storage.build_search_index()

In [ ]:
banned_sentences = dict((word,[]) for word in banned_words)


for idx, entry in gab_posts.iterrows():
  body = entry['body']
  for word in banned_words:
    if word in body:
      # Can just include sentence if we want
      banned_sentences[word].append(body)

In [ ]:
final_count = {}
for banned_word, sentences in banned_sentences.items():
  ctr_passed_sentences = 0
  ctr_total_checked = 0
  final_count[banned_word] = {}
  for sentence in sentences:
    if ctr_passed_sentences > 100 or ctr_total_checked > 1000:
      break
    ctr_total_checked = ctr_total_checked + 1
    try:
      distances, neighbors, contexts = storage.query(query_sent=sentence, query_word=banned_word, k=5, filter_same_word=True)
      # print('BANNED WORD: {} \n ORIGINAL SENTENCE: {} '.format(banned_word, sentence))
      for w in neighbors:
          if w in final_count[banned_word]:
            final_count[banned_word][w] = final_count[banned_word][w] + 1
          else:
            final_count[banned_word][w] = 1
      # for d, w, c in zip(distances, neighbors, contexts):
      ctr_passed_sentences = ctr_passed_sentences + 1
          # print('{} {}  {}'.format(w, d, c.strip()))
    except Exception as e:
      continue

In [ ]:
finalized_top_10 = {}

for key, occur_dict in final_count.items():
  if len(occur_dict.keys()) > 0:
    # Sort occurances
    sorted_dict = dict(sorted(occur_dict.items(), key=lambda item: item[1], reverse=True))
    # Take top 10
    finalized_top_10[key] = {k: sorted_dict[k] for k in list(sorted_dict)[:10]}
  else:
    finalized_top_10[key] = {}

In [ ]:
finalized_top_10
import json
with open('top_10_36000.json', 'w', encoding='utf-8') as f:
    json.dump(finalized_top_10, f, ensure_ascii=False, indent=4)

In [ ]:
finalized_top_10


{'pussie': {},
 'phonesex': {},
 'footjob': {},
 'horniest': {},
 'clitoris': {},
 'headfuck': {},
 'areola': {},
 'pussies': {'faggot': 3,
  'fags': 2,
  'titties': 1,
  'wussies': 1,
  'buttercups': 1,
  'pukes': 1,
  'perverts': 1},
 'goddamnes': {},
 'suicide': {'death': 11,
  'murder': 11,
  'die': 10,
  'rape': 9,
  'committing': 6,
  'treason': 5,
  'suicidal': 3,
  'commit': 2,
  'jihadis': 2,
  'committed': 2},
 'voyeurweb': {},
 'suicide girls': {},
 'niggarded': {},
 'deepthroat': {},
 'fuckbuddy': {},
 'nigra': {},
 'freefuck': {},
 'boob': {'blowjob': 2, 'booger': 1, 'boop': 1, 'douche': 1},
 'hentai': {'futanari': 4,
  'futa': 4,
  'bisokuzenshin': 4,
  'kasumi': 3,
  'ecchi': 2,
  'hirono': 1,
  'yori': 1,
  'arigato': 1},
 'rentafuck': {},
 'wanking': {},
 'jerk off': {},
 'molester': {'rapist': 4,
  'molestors': 3,
  'molested': 1,
  'molestation': 1,
  'molesting': 1},
 'horney': {},
 'titfuckin': {},
 'milf': {},
 'wrapping men': {},
 'whorefucker': {},
 'masturbatin

In [ ]:
print('hello')

hello


# query homonymous words

Now see how it works: 

* if the word "bank" is in context of "power bank", then the nearest neighbor is a "power bank" as well.

* if the word "bank" is in context of "investment bank", then the nearest neighbor is a "bank" as well, but in financial context



In [ ]:
distances, neighbors, contexts = storage.query(query_sent='It is a power bank.', query_word='bank', k=5)
for d, w, c in zip(distances, neighbors, contexts):
    print('{} {}  {}'.format(w, d, c.strip()))

In [ ]:
distances, neighbors, contexts = storage.query(query_sent='It is an investment bank.', query_word='bank', k=5)
for d, w, c in zip(distances, neighbors, contexts):
    print('{} {}  {}'.format(w, d, c.strip()))

If we look for the neighbors not containing the word "bank", then with investment context it is all about finance, but for "power bank" there are a few non-financial contexts. 

Probably, with a larger corpus, we would be able to find even more relevant examples (like "battery")

In [ ]:
distances, neighbors, contexts = storage.query(query_sent='It is an investment bank.', query_word='bank', k=5, filter_same_word=True)
total = 0
for d, w, c in zip(distances, neighbors, contexts):
    print('{} {}  {}'.format(w, d, c.strip()))


In [ ]:
distances, neighbors, contexts = storage.query(query_sent='It is a power bank.', query_word='bank', k=5, filter_same_word=True)
total = 0
for d, w, c in zip(distances, neighbors, contexts):
    print('{} {}  {}'.format(w, d, c.strip()))

For a "river bank", there are no relevant examples in our small corpus (10k sentences only), but the result is still not completely meaningless (e.g. "river side" is related to "river bank").

In [ ]:
distances, neighbors, contexts = storage.query(query_sent='It is a river bank.', query_word='bank', k=5, filter_same_word=True)
total = 0
for d, w, c in zip(distances, neighbors, contexts):
    print('{} {}  {}'.format(w, d, c.strip()))

# query named entities

Now let's try a query with named entity. We can see that Amazon the company and Amazon the toponym have nearest neighbors which are a company and a toponym as well. 

In [ ]:
s = "Bezos announced that its two-day delivery service, Amazon Prime, had surpassed 100 million subscribers worldwide."
distances, neighbors, contexts = storage.query(query_sent=s, query_word='amazon', k=5)
for d, w, c in zip(distances, neighbors, contexts):
    print('{} {}  {}'.format(w, d, c.strip()))

In [ ]:
s = "The Atlantic has sufficient wave and tidal energy to carry most of the Amazon's sediments out to sea, thus the river does not form a true delta"
distances, neighbors, contexts = storage.query(query_sent=s, query_word='amazon', k=5)
for d, w, c in zip(distances, neighbors, contexts):
    print('{} {}  {}'.format(w, d, c.strip()))

Moreover, we can infer that Amazon the company is related to Google, Alexa and Netflix, whereas Amazon the river is related to Brazil and the Brazilian city Belem. 

In [ ]:
s = "Bezos announced that its two-day delivery service, Amazon Prime, had surpassed 100 million subscribers worldwide."
distances, neighbors, contexts = storage.query(query_sent=s, query_word='amazon', k=5, filter_same_word=True)
for d, w, c in zip(distances, neighbors, contexts):
    print('{} {}  {}'.format(w, d, c.strip()))

In [ ]:
s = "The Atlantic has sufficient wave and tidal energy to carry most of the Amazon's sediments out to sea, thus the river does not form a true delta"
distances, neighbors, contexts = storage.query(query_sent=s, query_word='amazon', k=5, filter_same_word=True)
for d, w, c in zip(distances, neighbors, contexts):
    print('{} {}  {}'.format(w, d, c.strip()))